# Quick Demonstration for Global calibration of safety and combination factors

Calibrate a set of safety and combination factors for two different structures identified by two different LSFs.

The first LSF is given by:
$$ z_1*R \geq (0.4*G + 0.6*Q1 + 0.3*Q2)$$

The second LSF is given by:
$$ z_2*R \geq (0.4*G + 0.3*Q1 + 0.6*Q2)$$

Note:
1. I think it is reasonable to assume that any global calibration exercise would contain the same load effects albeit with different nominal values.
2. This is just a quick demonstration.
3. The objective is also to utilize as much as existing Pystra code as possible.

## Import Libraries and define utility functions

In [1]:

import pystra as ra
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
def lsf(z, R, G, Q1, Q2, cg, c1, c2):
    return z*R - (cg*G + c1*Q1 + c2*Q2)

In [3]:
def create_df_gamma(gg, g1, g2):
    """
    Create a Dataframe of Gamma factors from a set of input values such that
    it can be utilized by the respective calibration object.

    """
    list_g = [gg, g1, g2]
    cols = ['G', 'Q1', 'Q2']
    idx = ['Q1_max', 'Q2_max']
    df = pd.DataFrame(data=[list_g, list_g], columns=cols, index=idx)
    return df

def create_df_phi(phi):
    """
    Create a Dataframe of Phi factors from a set of input values such that
    it can be utilized by the respective calibration object.

    """
    list_phi = [phi]
    cols = ['R']
    idx = ['Q1_max', 'Q2_max']
    df = pd.DataFrame(data=[list_phi, list_phi], columns=cols, index=idx)
    return df
    
def create_df_psi(psi1, psi2):
    """
    Create a Dataframe of Psi factors from a set of input values such that
    it can be utilized by the respective calibration object.

    """
    list_psi1 = [1, 1, psi2]
    list_psi2 = [1, psi1, 1]
    cols = ['G', 'Q1', 'Q2']
    idx = ['Q1_max', 'Q2_max']
    df = pd.DataFrame(data=[list_psi1, list_psi2], columns=cols, index=idx)
    return df

In [4]:
def run_calib_design(dfphi, dfgamma, dfpsi, calib):
    """
    Take a set of factors for a calibration object and run a design check.

    """
    calib.df_phi = dfphi
    calib.df_gamma = dfgamma
    calib.df_psi = dfpsi
    design_z = calib.get_design_param_factor()
    design_beta = calib.calc_beta_design_param(np.max(design_z))
    return design_beta

In [5]:
def obj_func_global(list_factors):
    """
    Objective function for minimization: weighted mean sequared error.
    Taken from Gayton (2004) pg. 5.

    Takes a set of input parameters, creates corresponding dataframes,
    finds corresponding design reliabilities for each calibration object,
    and returns the residual.
    """
    print(list_factors)
    gg, g1, g2, psi1, psi2, phi = list_factors
    df_phi = create_df_phi(phi)
    df_gamma = create_df_gamma(gg, g1, g2)
    df_psi = create_df_psi(psi1, psi2)
    design_beta1 = run_calib_design(df_phi, df_gamma, df_psi, calib1)
    design_beta2 = run_calib_design(df_phi, df_gamma, df_psi, calib2)
    list_betaj = np.concatenate([design_beta1, design_beta2])
    list_wj = np.ones(4)
    res = np.sum([xx * (yy-betaT)**2 for xx,yy in zip(list_wj, list_betaj)])
    return res

## Set up problem input

In [6]:
## Set parameters for LoadCombination: Distributions and combinations
# Annual max distributions
Q1max = ra.Gumbel("Q1", 1, 0.2)  # Imposed Load
Q2max = ra.Gumbel("Q2", 1, 0.4)  # Wind Load
# Parameters of inferred point-in-time parents
Q1pit = ra.Gumbel("Q1", 0.89, 0.2)  # Imposed Load
Q2pit = ra.Gumbel("Q2", 0.77, 0.4)  # Wind Load
Q_dict = {'Q1': {'max': Q1max, 'pit': Q1pit},
          'Q2': {'max': Q2max, 'pit': Q2pit}}
# Constant values
cg1 = ra.Constant("cg", 0.4)
c11 = ra.Constant("c1", 0.6)
c21 = ra.Constant("c2", 0.3)
cg2 = ra.Constant("cg", 0.4)
c12 = ra.Constant("c1", 0.3)
c22 = ra.Constant("c2", 0.6)
z = ra.Constant("z", 1)  # Design parameter for resistance with arbitrary default value

## Define other random variables
Rdist = ra.Lognormal("R", 1.0, 0.15)  # Resistance
Gdist = ra.Normal("G", 1, 0.1)  # Permanent Load (other load variable)
loadcombinations = {'Q1_max':['Q1'], 'Q2_max':['Q2']}

In [7]:
## Set parameters for calibration: nominal values and target Beta
Qk = np.array([Q1max.ppf(0.98), Q2max.ppf(0.98)])
Gk = np.array([Gdist.ppf(0.5)])
Rk = np.array([Rdist.ppf(0.05)])
rvs_all = ['R', 'G', 'Q1', 'Q2', 'Q3']
dict_nom = dict(zip(rvs_all, np.concatenate([Rk, Gk, Qk])))
betaT = 4.3

## Instantiate Combination and Calibration Objects

In [8]:
## Instantiate LoadCombination object
lc1 = ra.LoadCombination(lsf, Q_dict, [Gdist], [Rdist], [z, cg1, c11, c21],
                      dict_comb_cases=loadcombinations)
lc2 = ra.LoadCombination(lsf, Q_dict, [Gdist], [Rdist], [z, cg2, c12, c22],
                      dict_comb_cases=loadcombinations)


## Instantiate Calibration object
calib1 = ra.Calibration(lc1, target_beta=betaT, dict_nom_vals=dict_nom,
                        calib_var='z', est_method="matrix", 
                        calib_method="optimize", print_output=False)

calib2 = ra.Calibration(lc2, target_beta=betaT, dict_nom_vals=dict_nom,
                        calib_var='z', est_method="matrix", 
                        calib_method="optimize", print_output=False)

## Global Optimization

In [9]:
## Run Global optimization
# Starting values taken from results of calib1.run()
start = [1.0371, 1.069218, 1.102563, 0.931848, 0.898165, 0.846876]
# Specify Bounds for the respective parameters
g_low = 1.0
g_up = 1.6
psi_low = 0.1
psi_up = 1.0
phi_low = 0.5
phi_up = 1.0
bnds = ((g_low, g_up), (g_low, g_up), (g_low, g_up), (psi_low, psi_up), 
        (psi_low, psi_up), (phi_low, phi_up))
# Optimize
result = minimize(obj_func_global, x0=start, bounds=bnds)

[1.0371   1.069218 1.102563 0.931848 0.898165 0.846876]
[1.03710001 1.069218   1.102563   0.931848   0.898165   0.846876  ]
[1.0371     1.06921801 1.102563   0.931848   0.898165   0.846876  ]
[1.0371     1.069218   1.10256301 0.931848   0.898165   0.846876  ]
[1.0371     1.069218   1.102563   0.93184801 0.898165   0.846876  ]
[1.0371     1.069218   1.102563   0.931848   0.89816501 0.846876  ]
[1.0371     1.069218   1.102563   0.931848   0.898165   0.84687601]
[1.6      1.6      1.6      1.       0.898165 0.5     ]
[1.59999999 1.6        1.6        1.         0.898165   0.5       ]
[1.6        1.59999999 1.6        1.         0.898165   0.5       ]
[1.6        1.6        1.59999999 1.         0.898165   0.5       ]
[1.6        1.6        1.6        0.99999999 0.898165   0.5       ]
[1.6        1.6        1.6        1.         0.89816501 0.5       ]
[1.6        1.6        1.6        1.         0.898165   0.50000001]
[1.07781626 1.10761107 1.13854412 0.93677764 0.898165   0.82178541]
[1.0

### Store results

In [10]:
factors_opt = result.x

opt_res = obj_func_global(factors_opt)
df_phi_opt = create_df_phi(factors_opt[5])
df_gamma_opt = create_df_gamma(factors_opt[0], factors_opt[1], factors_opt[2])
df_psi_opt = create_df_psi(factors_opt[3], factors_opt[4])
design_beta_1_opt = run_calib_design(df_phi_opt, df_gamma_opt, df_psi_opt, calib1)
design_beta_2_opt = run_calib_design(df_phi_opt, df_gamma_opt, df_psi_opt, calib1)

[1.         1.         1.50457124 0.71738177 0.72367792 0.83872794]


### Display results

In [11]:
# Display results
print(f'\n Phi optimal = \n{df_phi_opt}')
print(f'\n Gamma optimal = \n{df_gamma_opt}')
print(f'\n Psi optimal = \n{df_psi_opt}')
print(f'\nOptimal design reliabilities for problem 1: {design_beta_1_opt}')
print(f'\nOptimal design reliabilities for problem 2: {design_beta_2_opt}')



 Phi optimal = 
               R
Q1_max  0.838728
Q2_max  0.838728

 Gamma optimal = 
          G   Q1        Q2
Q1_max  1.0  1.0  1.504571
Q2_max  1.0  1.0  1.504571

 Psi optimal = 
        G        Q1        Q2
Q1_max  1  1.000000  0.723678
Q2_max  1  0.717382  1.000000

Optimal design reliabilities for problem 1: [4.30323642 4.29676697]

Optimal design reliabilities for problem 2: [4.30323642 4.29676697]


Note that the optimal design reliabilities can be slightly less than the target reliability because there is no constraint on setting $\beta_j\geq \beta_T$